# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,papatowai,-46.5619,169.4708,291.08,54,100,1.23,NZ,1698197244
1,1,waitangi,-43.9535,-176.5597,287.49,93,100,3.13,NZ,1698197245
2,2,bredasdorp,-34.5322,20.0403,285.01,82,15,0.97,ZA,1698197245
3,3,grytviken,-54.2811,-36.5092,273.97,93,87,4.78,GS,1698197245
4,4,port elizabeth,-33.9180,25.5701,287.77,86,0,4.63,ZA,1698197245


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(x='Lng', y='Lat',  c='Humidity', 
                                      hover_cols=['City','Humidity'], tiles='OSM', colormap='viridis',
                                      geo=True,   size='Humidity', height=500, width=800, colorbar=True, legend= "right")




 
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_data_df = city_data_df[(city_data_df["Max Temp"] <= 300) & (city_data_df["Max Temp"] > 295) & 
                             (city_data_df["Wind Speed"] < 10) & city_data_df["Cloudiness"] <20]


# Drop any rows with null values
ideal_data_df.dropna()

# Display sample data
ideal_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,papatowai,-46.5619,169.4708,291.08,54,100,1.23,NZ,1698197244
1,1,waitangi,-43.9535,-176.5597,287.49,93,100,3.13,NZ,1698197245
2,2,bredasdorp,-34.5322,20.0403,285.01,82,15,0.97,ZA,1698197245
3,3,grytviken,-54.2811,-36.5092,273.97,93,87,4.78,GS,1698197245
4,4,port elizabeth,-33.9180,25.5701,287.77,86,0,4.63,ZA,1698197245
...,...,...,...,...,...,...,...,...,...,...
558,558,camiri,-20.0500,-63.5167,295.04,89,100,1.76,BO,1698197385
559,559,gilmer,32.7287,-94.9424,298.38,53,0,2.57,US,1698197386
560,560,zapata,26.9073,-99.2714,303.21,51,0,6.69,US,1698197359
561,561,aykhal,66.0000,111.5000,261.99,100,99,4.90,RU,1698197386


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= pd.DataFrame(city_data_df, columns=["City", "Country", "Lat", "Lng", "Humidity"])


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,papatowai,NZ,-46.5619,169.4708,54,
1,waitangi,NZ,-43.9535,-176.5597,93,
2,bredasdorp,ZA,-34.5322,20.0403,82,
3,grytviken,GS,-54.2811,-36.5092,93,
4,port elizabeth,ZA,-33.9180,25.5701,86,
...,...,...,...,...,...,...
558,camiri,BO,-20.0500,-63.5167,89,
559,gilmer,US,32.7287,-94.9424,53,
560,zapata,US,26.9073,-99.2714,51,
561,aykhal,RU,66.0000,111.5000,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_type = "hotel"
radius = 10000 

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
# Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    
    params = {"categories": "accommodation.hotel",   # This category includes hotels
              "filter": f"circle:{lng},{lat},{10000}",      # We further filter to only get hotels
              "limit": 1,   
              "apiKey": geoapify_key}

    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
 
    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
   
    
    # Convert the API response to JSON format
    name_address = name_address.json()
   
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except Exception as ex:# (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display sample data
hotel_df

Starting hotel search
papatowai - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
bredasdorp - nearest hotel: Victoria Hotel
grytviken - nearest hotel: No hotel found
port elizabeth - nearest hotel: Garden Court Kings Beach
tiksi - nearest hotel: Арктика
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
celestun - nearest hotel: Cabañas Cinco Elementos
hotan - nearest hotel: يۈدۇ مېھمانخانىسى
ancud - nearest hotel: Mirador Quempillen
carbonear - nearest hotel: No hotel found
kawambwa - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
kavaratti - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
east london - nearest hotel: Luneburgh Cottages
arteche - nearest hotel: No hotel found
whitehorse - nearest hotel: Airport Chalet
taybad - nearest hotel: No hotel found
aasiaat - nearest hotel: SØMA
talcahuano - nearest hotel: Wyndham Concepción Pettra
mikuni - nearest h

silver springs - nearest hotel: No hotel found
avarua - nearest hotel: Natura Inano Villa
champerico - nearest hotel: Posada Del Mar
pangody - nearest hotel: Газпром энерго
porto novo - nearest hotel: SODAS
shengli - nearest hotel: 蓝海新悦大酒店
axim - nearest hotel: Lou Moon lodge
san julian - nearest hotel: No hotel found
skagen - nearest hotel: Ruths Hotel
chingola - nearest hotel: Protea Hotel
abra pampa - nearest hotel: Residencia El Norte
tarawa - nearest hotel: The George Hotel Kiribati Hotel
cabedelo - nearest hotel: Espaço Soleil Pousada
kargasok - nearest hotel: Северянка
mejit - nearest hotel: No hotel found
sretensk - nearest hotel: No hotel found
namibe - nearest hotel: Infotur Namibe
jamestown - nearest hotel: Chautauqua Harbor Hotel
annaba - nearest hotel: Hôtel El Mountazah
kristiansund - nearest hotel: Thon Hotel Storgata
gondar - nearest hotel: Taye Belay Hotel
belogorsk - nearest hotel: гостиница Восток
ascension - nearest hotel: Days Inn
shenjiamen - nearest hotel: Hilton

labuan - nearest hotel: OYO 969 Hotel Pantai View
maintirano - nearest hotel: Hotel Taratra
ghat - nearest hotel: فندق تاسيلي
orosei - nearest hotel: CLUB HOTEL MARINA BEACH
moissala - nearest hotel: No hotel found
cozumel - nearest hotel: Coral Princess Hotel & Dive Resort
karratha - nearest hotel: Karratha International Hotel
temiskaming shores - nearest hotel: Waterfront Inn
lebanon - nearest hotel: Fairfield Inn & Suites by Marriott Lebanon
new glasgow - nearest hotel: Holiday Inn Express
barcelos - nearest hotel: Hotel do Terço
tauranga - nearest hotel: Old Tauranga Post Office
praia da vitoria - nearest hotel: Aparthotel Atlântida Mar Hotel
galesong - nearest hotel: No hotel found
inarajan village - nearest hotel: No hotel found
tabas - nearest hotel: مهمانسرای فرمانداری
bonthe - nearest hotel: No hotel found
sidmouth - nearest hotel: Woodlands Hotel
leava - nearest hotel: Somalama
sabha - nearest hotel: بيت سبها للشقق الفندقية
kupjansk - nearest hotel: Піраміда
live oak - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
0,papatowai,NZ,-46.5619,169.4708,54,No hotel found
1,waitangi,NZ,-43.9535,-176.5597,93,Hotel Chathams
2,bredasdorp,ZA,-34.5322,20.0403,82,Victoria Hotel
3,grytviken,GS,-54.2811,-36.5092,93,No hotel found
4,port elizabeth,ZA,-33.9180,25.5701,86,Garden Court Kings Beach
...,...,...,...,...,...,...
558,camiri,BO,-20.0500,-63.5167,89,Sol Chaqueño
559,gilmer,US,32.7287,-94.9424,53,No hotel found
560,zapata,US,26.9073,-99.2714,51,Sunbridge Inn & Suites
561,aykhal,RU,66.0000,111.5000,100,Уют


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(x='Lng', y='Lat',  c='Humidity', 
                                    hover_cols=['City', 'Hotel Name',  'Humidity'], 
                                    tiles='OSM', colormap='viridis', geo=True,   
                                    size='Humidity', height=500, width=800, colorbar=True, legend= "right")


# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name)